In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [7]:
kospi = pd.read_csv('kospi_add_sector_2024-12-28.csv')
kosdaq = pd.read_csv('kosdaq_add_sector_2024-12-28.csv')

In [8]:
# n개월 전 날짜 계산 함수(개월단위)
def calculate_start_date(months_ago, end_date):
    start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - relativedelta(months=months_ago)
    return start_date.strftime('%Y-%m-%d')

# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')

In [12]:
def combined_stock_analysis(market='KOSPI', month_ago=1, end_date=today_str):
    """
    주어진 시장(KOSPI 또는 KOSDAQ)의 각 종목에 대한 변동성과 수익률을 계산하는 함수.

    Parameters:
    market (str): 'KOSPI' 또는 'KOSDAQ' 중 하나를 선택하여 해당 시장의 종목을 대상으로 분석.
    month_ago (int): 계산할 시작 날짜로부터 몇 개월 전부터 수익률을 계산할 것인지 지정.
    end_date (str, optional): 데이터 조회의 종료 날짜 (형식: 'YYYY-MM-DD'). 기본값은 오늘 날짜.

    Returns:
    pd.DataFrame: 각 종목의 코드, 이름, 변동성, 총 수익률, 평균 수익률 정보를 포함하는 데이터프레임.
    
    예외 처리:
    데이터 로드 중 오류가 발생하면 해당 종목에 대한 오류 메시지를 출력.
    """
    
    start_date = calculate_start_date(month_ago, end_date)
    results = []  # 결과를 저장할 리스트

    # 시장 데이터 선택
    if market == 'KOSPI':
        sector_data = kospi
    elif market == 'KOSDAQ':
        sector_data = kosdaq
    else:
        raise ValueError(" 'KOSPI', 'KOSDAQ' 두 시장만 지원하는 기능입니다.")
    
    # 'Code'를 인덱스로 설정
    sector_data = sector_data.set_index('Code')

    # 각 종목의 데이터를 가져와 변동성과 수익률 계산
    for index, row in sector_data.iterrows():
        ticker = index
        try:
            # 각 종목의 데이터 가져오기
            data = fdr.DataReader(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"종목 {ticker}의 데이터가 없습니다. 건너뜁니다.")
                continue
            
            # 수익률 계산
            
            data['Returns'] = data['Close'].pct_change() * 100  # 수익률을 퍼센트로 변환
            data.dropna(inplace=True)  # NaN 값 제거
            
            if len(data) == 1:  # 상장이후 일일수익률 데이터가 단 하나라  수익률 계산이 의미가 없는경우
                print(f"종목 {ticker}은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.")
                continue
      
            if len(data) == 0: # 상장이후 종가가 하나라 수익률 계산 자체가 되지 않는경우
                print(f"종목 {ticker}의 데이터가 충분하지 않습니다. 건너뜁니다.")
                continue
            
            # 변동성 계산 (표준편차)
            volatility = data['Returns'].std()
            
            # 총 수익률 계산
            total_return = (data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0] * 100
            
            # 평균 수익률 계산산
            avg_return = data['Returns'].mean()
            

            # 결과 저장
            results.append({
                'Code': ticker,                 # 종목코드
                'Name': row['Name'],            # 종목명
                'Volatility': volatility,       # 변동성(%)
                'TotalReturn': total_return,    # 총수익률(%)
                'AvgReturn': avg_return,        # 평균수익률(%)      
            })

        except Exception as e:
            print(f"{ticker} 데이터 오류: {e}")

    # 결과 DataFrame 생성
    results_df = pd.DataFrame(results)

    return results_df


In [13]:
kospi_df = combined_stock_analysis(market='KOSPI')
kospi_df.head()

,Code,Name,Volatility,TotalReturn,AvgReturn
0,005930,삼성전자,1.620596,-0.922509,-0.152225
1,000660,SK하이닉스,2.300046,9.130707,0.425609
2,373220,LG에너지솔루션,2.717703,-8.781127,-0.690052
3,207940,삼성바이오로직스,1.389048,-4.205128,-0.331636
4,005380,현대차,2.109992,-1.601831,-0.071277
